In [ ]:
import os
import skspeech

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Some plotting niceties
plt.rc('figure', figsize=(10, 4))
sns.set_style('white')
sns.set_style('ticks')

import nengo
from nengo import spa
from nengo.spa import Vocabulary
import nengo_gui.ipython
from nengo.dists import Choice, Uniform, ClippedExpDist

import phd
from phd.networks import dmp

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

In [ ]:
# Create our semantic pointers
dimensions = 96
rng = np.random.RandomState(0)
vocab = Vocabulary(dimensions=dimensions, rng=rng, unitary=['NEXTPOS', 'POS1'])
vocab.parse('NEXTPOS')
vocab.parse('POS1')
for i in xrange(2, 9):
    vocab.add('POS%d' % i, vocab.parse('POS%d * NEXTPOS' % (i-1)))

syllables = ('PAT', 'DAS', 'KAP')
for syll in syllables:
    vocab.parse(syll)

just_syllables = vocab.create_subset(syllables)

In [ ]:
# Get our gesture trajectories / functions

def get_traj(path, dt=0.001):
    gs = skspeech.vtl.parse_ges(path)
    wavfile = os.path.basename(path)[:-4] + '.wav'
    if not os.path.exists(wavfile):
        gs.synthesize(wavfile=wavfile)
    traj = gs.trajectory(dt=dt)
    return traj

def rescale(val, old_min, old_max, new_min, new_max):
    old_range = old_max - old_min
    new_range = new_max - new_min
    return (((val - old_min) * new_range) / old_range) + new_min

def traj2func(traj, dt=0.001):
    traj_ix = np.unique(np.nonzero(traj)[1])
    traj = traj[:, traj_ix]
    t_end = traj.shape[0] * dt
    def _trajf(x):
        # x goes from 0 to 1; normalize by t_end
        # But give a bit of 0 at the start...
        if x < 0.1:
            return np.zeros(traj.shape[1])
        ix = min(int(rescale(x, 0.1, 1., 0., t_end-0.1) / dt), traj.shape[0]-1)
        return traj[ix]
    _trajf.traj = traj
    _trajf.ix = traj_ix
    return _trajf, traj_ix

dt = 0.001
pat = get_traj(phd.ges_path('ges-de-cvc', 'pat.ges'), dt)
das = get_traj(phd.ges_path('ges-de-cvc', 'das.ges'), dt)
kap = get_traj(phd.ges_path('ges-de-cvc', 'kap.ges'), dt)
pat_f, pat_ix = traj2func(pat, dt)
das_f, das_ix = traj2func(das, dt)
kap_f, kap_ix = traj2func(kap, dt)

In [ ]:
# 1: SPA sequence stuff

difference_gain = 15
neurons_per_d = 50

with spa.SPA() as model:
    model.sequence = spa.State(dimensions=dimensions, neurons_per_dimension=neurons_per_d)

    model.syll_idx = nengo.networks.InputGatedMemory(
        neurons_per_d, dimensions=dimensions, difference_gain=difference_gain)
    model.syll_idx_next = nengo.networks.InputGatedMemory(
        neurons_per_d, dimensions=dimensions, difference_gain=difference_gain)
    nengo.Connection(model.syll_idx.output, model.syll_idx_next.input)
    nengo.Connection(model.syll_idx_next.output, model.syll_idx.input,
                     transform=vocab['NEXTPOS'].get_convolution_matrix())
    
    # syll = sequence * ~syll_idx
    model.bind = nengo.networks.CircularConvolution(
        neurons_per_d, dimensions, invert_a=False, invert_b=True)
    nengo.Connection(model.sequence.output, model.bind.A)
    nengo.Connection(model.syll_idx.output, model.bind.B)

    model.bind_next = nengo.networks.CircularConvolution(
        neurons_per_d, dimensions, invert_a=False, invert_b=True)
    nengo.Connection(model.sequence.output, model.bind_next.A)
    nengo.Connection(model.syll_idx_next.output, model.bind_next.B,
                     transform=vocab['NEXTPOS'].get_convolution_matrix())

    # clean up noisy syllable representations
    model.syllable = spa.AssociativeMemory(
        just_syllables, wta_output=True, threshold_output=True)
    nengo.Connection(model.bind.output, model.syllable.input)
    model.syllable_next = spa.AssociativeMemory(
        just_syllables, wta_output=True, threshold_output=True)
    # XXX add default output vectors here!
    nengo.Connection(model.bind_next.output, model.syllable_next.input)

    # Start the system off
    init_idx = nengo.Node(
        lambda t: vocab.parse('POS1').v if t < 0.3 else vocab.parse('0').v)
    nengo.Connection(init_idx, model.syll_idx.input)
    nengo.Connection(nengo.Node(vocab.parse('PAT*POS1 + DAS*POS2 + KAP*POS3').v),
                     model.sequence.input)

In [ ]:
# with model:
#     nengo.Connection(nengo.Node(lambda t: 1. if t % 0.4 < 0.2 else 0.), model.gate)
#     p_1 = nengo.Probe(model.syllable.output, synapse=0.01)
#     p_2 = nengo.Probe(model.syllable_next.output, synapse=0.01)

# sim = nengo.Simulator(model)
# sim.run(1.4)

# plt.plot(sim.trange(), 
#          spa.similarity(sim.data[p_1],
#                         vocab.create_subset(syllables)))
# plt.legend(syllables, loc='best')
# plt.figure()
# plt.plot(sim.trange(), 
#          spa.similarity(sim.data[p_2],
#                         vocab.create_subset(syllables)))
# plt.legend(syllables, loc='best')

In [ ]:
# 2: Timing network

def kick_f(x):
    # We'll do this as a function of x, like the forcing_f
    # return 1 if x < 0.1 or x > 0.9 else 0.
    return 1 if x > 0.7 else 0.

def gate_f(x):
    # Like kick_f, but for switching to the next syllable
    # When we flip up the gating signal, the next syllable
    # is loaded in the second working memory,
    # and so it will be disinhibited.
    # But that's fine, because
    # it won't be activated until the kick.
    # Once the kick happens, the gating signal
    # goes down, which means both memories
    # will go back to representing the same current syllable.
    return 1 if 0.35 < x < 0.75 else 0.

# By default, timer will progress at 2 Hz
tau = 0.05
omega = 2 * tau * 2 * np.pi

with model:
    dmp_in = nengo.Ensemble(200, dimensions=2)
    timer = nengo.Ensemble(200, dimensions=2)
    nengo.Connection(dmp_in, timer)

    # The kick resets the DMP oscillators
    kick = nengo.Ensemble(60, dimensions=1,
                          encoders=Choice([[1]]),
                          intercepts=ClippedExpDist(0.15, 0.5, 1))
    nengo.Connection(timer, kick, function=dmp.radial_f(kick_f))
    # There's an issue in that the kick forces the DMP oscillator
    # to a specific point in the state space,
    # but that's the exact point where the kick is active, so
    # it will just stay there. Therefore, when the kick is active,
    # we let the timer oscillate on its own
    timer_recurr = nengo.Ensemble(200, dimensions=2)
    nengo.Connection(timer, timer_recurr)
    nengo.Connection(timer_recurr, timer,
                     synapse=tau,
                     transform=[[1, -omega], [omega, 1]])

    # kick disinhibiting the recurrence
    tr_inhibit = nengo.Ensemble(20, dimensions=1,
                                intercepts=ClippedExpDist(0.15, -0.5, 0.1),
                                encoders=Choice([[1]]))
    nengo.Connection(tr_inhibit.neurons, timer_recurr.neurons,
                     transform=-np.ones((200, 20)))
    nengo.Connection(kick, tr_inhibit, transform=-1)

    # The gate switches the working memory representations
    gate = nengo.Ensemble(60, dimensions=1,
                          encoders=Choice([[1]]),
                          intercepts=ClippedExpDist(0.15, 0.4, 1))
    nengo.Connection(gate, model.syll_idx.gate)
    # bias so that model.syll_idx_next.gate gets 1 - gate
    nengo.Connection(nengo.Node(1, label='gate bias'), model.syll_idx_next.gate)
    nengo.Connection(gate, model.syll_idx_next.gate, transform=[-1])

#     nengo.Connection(timer, gate, function=dmp.radial_f(gate_f))
    # Let's just have gate always on, but inhibited by the kick
    gate_bias = nengo.Node(output=1)
    nengo.Connection(gate_bias, gate)
#     nengo.Connection(kick.neurons, gate.neurons, transform=-np.ones((60, 60)))
    nengo.Connection(kick, gate, transform=-0.7)

    # Something to do with disinhibiting things...?

    # Find a better way?
    init_kick = nengo.Node(output=lambda t: 1.0 if 0.1 < t < 0.2 else 0.0)
    nengo.Connection(init_kick, kick)
    nengo.Connection(init_kick, timer, transform=[[-1], [0]])
    init_gate = nengo.Node(output=lambda t: -1.0 if t < 0.2 else 0.0)
    nengo.Connection(init_gate, gate)

In [ ]:
# 3: Trajectory DMP stuff
with model:
    pat_dmp = dmp.RhythmicDMP(100, freq=2.5, forcing_f=pat_f)
    das_dmp = dmp.RhythmicDMP(100, freq=4.7, forcing_f=das_f)
    kap_dmp = dmp.RhythmicDMP(100, freq=3.2, forcing_f=kap_f)

    speech = nengo.networks.EnsembleArray(60, pat.shape[1],
        encoders=Choice([[1]]), intercepts=ClippedExpDist(0.15, 0.2, 1))
    nengo.Connection(pat_dmp.output, speech.input[pat_ix])
    nengo.Connection(das_dmp.output, speech.input[das_ix])
    nengo.Connection(kap_dmp.output, speech.input[kap_ix])

    # Connect to the timer
    nengo.Connection(pat_dmp.osc, dmp_in)
    nengo.Connection(das_dmp.osc, dmp_in)
    nengo.Connection(kap_dmp.osc, dmp_in)
    nengo.Connection(kick, pat_dmp.reset)
    nengo.Connection(kick, das_dmp.reset)
    nengo.Connection(kick, kap_dmp.reset)

    # Our associative memories ensembles will disinhibit the appropriate dmps
    for curr_ens, next_ens, s_dmp in zip(model.syllable.am.thresh_ens.ea_ensembles,
                                         model.syllable_next.am.thresh_ens.ea_ensembles,
                                         [pat_dmp, das_dmp, kap_dmp]):
        # Order should be maintained
        nengo.Connection(curr_ens, s_dmp.disinhibit)
        nengo.Connection(next_ens, s_dmp.disinhibit)

In [ ]:
# 4: probes

with model:
    p_syll = nengo.Probe(model.syllable.output, synapse=0.01)
    p_gate = nengo.Probe(gate, synapse=0.01)
    p_kick = nengo.Probe(kick, synapse=0.01)
    p_timer = nengo.Probe(timer, synapse=0.01)
    p_pdis = nengo.Probe(pat_dmp.disinhibit, synapse=0.01)
    p_ddis = nengo.Probe(das_dmp.disinhibit, synapse=0.01)
    p_kdis = nengo.Probe(kap_dmp.disinhibit, synapse=0.01)
    p_pat = nengo.Probe(pat_dmp.osc, synapse=0.01)
    p_das = nengo.Probe(das_dmp.osc, synapse=0.01)
    p_kap = nengo.Probe(kap_dmp.osc, synapse=0.01)
    p_sa = nengo.Probe(speech.output, synapse=0.01)

In [ ]:
sim = nengo.Simulator(model)
sim.run(1.3)

In [ ]:
# plt.plot(sim.trange(),
#          spa.similarity(sim.data[p_syllidx],
#                         vocab.create_subset(['POS1', 'POS2', 'POS3'])))
# plt.legend(['POS1', 'POS2', 'POS3'], loc='best')
# plt.ylim([-0.2, 1.2])

# plt.figure()
plt.plot(sim.trange(), 
         spa.similarity(sim.data[p_syll],
                        vocab.create_subset(syllables)))
plt.legend(syllables, loc='best')

plt.figure()
plt.plot(sim.trange(), sim.data[p_gate])
plt.plot(sim.trange(), sim.data[p_kick])
plt.plot(sim.trange(), sim.data[p_timer])
plt.legend(['gate', 'kick', 'timer', 'timer'], loc="best")
# plt.ylim([-0.1, 1.1])

plt.figure()
plt.plot(sim.trange(), sim.data[p_pdis])
plt.plot(sim.trange(), sim.data[p_ddis])
plt.plot(sim.trange(), sim.data[p_kdis])
plt.legend(['PAT', 'DAS', 'DAP'], loc="best")
# plt.ylim([-0.1, 1.1])

plt.figure()
plt.subplot(2, 2, 1)
plt.plot(sim.data[p_timer].T[0], sim.data[p_timer].T[1])
plt.subplot(2, 2, 2)
plt.plot(sim.data[p_pat].T[0], sim.data[p_pat].T[1])
plt.subplot(2, 2, 3)
plt.plot(sim.data[p_das].T[0], sim.data[p_das].T[1])
plt.subplot(2, 2, 4)
plt.plot(sim.data[p_kap].T[0], sim.data[p_kap].T[1])

plt.figure()
plt.plot(sim.trange(), sim.data[p_sa]);

In [ ]:
# dmp = reload(dmp)
# with nengo.Network() as net:
#     pat_dmp = dmp.RhythmicDMP(200, freq=2.5, forcing_f=pat_f)
#     nengo.Connection(nengo.Node(0), pat_dmp.reset)
#     nengo.Connection(nengo.Node(1), pat_dmp.disinhibit)
# from nengo_gui.ipython import IPythonViz
# IPythonViz(model, cfg='tmp.cfg')